In [10]:
import asyncio
import aiohttp
import json
from pprint import pprint

from attraction_api import fetch_attractions
from hotel_api import HotelRetriever
from flight_api import FlightRetriever
from llm_retrieve import llm_retrieve
from llm_generate import llm_generate

In [11]:
with open("user_inputs.json", "r") as f:
    inputs = json.load(f)

trip_plan = llm_retrieve(inputs)
print("Loaded cities and parameters:")
for city, info in trip_plan.items():
    print(f" • {city}: Arrival={info['Arrival Date']}, Departure={info['Departure Date']}, Airport={info['Airport']}")


Loaded cities and parameters:
 • Srinagar: Arrival=2025-07-10, Departure=2025-07-13, Airport=SXR
 • Leh: Arrival=2025-07-13, Departure=2025-07-16, Airport=IXL
 • Chandigarh: Arrival=2025-07-16, Departure=2025-07-18, Airport=IXC
 • Shimla: Arrival=2025-07-18, Departure=2025-07-20, Airport=SLV
 • Dehradun: Arrival=2025-07-20, Departure=2025-07-22, Airport=DED
 • Jaipur: Arrival=2025-07-22, Departure=2025-07-25, Airport=JAI
 • Delhi: Arrival=2025-07-25, Departure=2025-07-28, Airport=DEL


In [12]:
async def gather_all():
    async with aiohttp.ClientSession() as session:
        attraction_tasks = [fetch_attractions(session, city) for city in trip_plan.keys()]
        attraction_results = await asyncio.gather(*attraction_tasks)

    hotel_results = HotelRetriever(trip_plan, numadults=2)
    flight_results = FlightRetriever(trip_plan)

    return attraction_results, hotel_results, flight_results


In [13]:
# Cell 4: Run & Merge
attractions, hotels, flights = await gather_all()

# Merge results by city
combined = {city: {"city": city, "attractions": [], "hotels": [], "flights": []} for city in trip_plan}

# Add attractions
for entry in attractions:
    combined[entry["city"]]["attractions"] = entry["attractions"]

# Add hotels
for entry in hotels:
    city = entry["city"]
    combined[city]["hotels"] = entry["hotels_by"]

# Add flights
for entry in flights:
    from_city = entry["from"]
    to_city = entry["to"]
    date = entry["date"]
    flights_data = entry.get("flights")
    note = entry.get("note")

    # Associate the flight with departure city
    combined[from_city].setdefault("flights", []).append({
        "to": to_city,
        "date": date,
        "flights": flights_data,
        "note": note
    })
    

[Attraction] 🔍 Fetching location_id for: Srinagar
[Attraction] 🔍 Fetching location_id for: Leh
[Attraction] 🔍 Fetching location_id for: Chandigarh
[Attraction] 🔍 Fetching location_id for: Shimla
[Attraction] 🔍 Fetching location_id for: Dehradun
[Attraction] 🔍 Fetching location_id for: Jaipur
[Attraction] 🔍 Fetching location_id for: Delhi
[Attraction] ✅ location_id for Dehradun = 297687
[Attraction] 🔍 Fetching attractions for Dehradun
[Attraction] ✅ location_id for Delhi = 304551
[Attraction] 🔍 Fetching attractions for Delhi
[Attraction] ✅ location_id for Srinagar = 297623
[Attraction] 🔍 Fetching attractions for Srinagar
[Attraction] 🌐 Raw results count for Dehradun: 20
[Attraction] 🎯 Returning 10 attractions for Dehradun
[Attraction] 🌐 Raw results count for Delhi: 20
[Attraction] 🎯 Returning 10 attractions for Delhi
[Attraction] 🌐 Raw results count for Srinagar: 20
[Attraction] 🎯 Returning 10 attractions for Srinagar
[Attraction] ✅ location_id for Shimla = 304552
[Attraction] 🔍 Fetchin

In [14]:
final_list = list(combined.values())

print("\n\u2705 After merging, sample output for first city:")
print(json.dumps(final_list[0], indent=2))


✅ After merging, sample output for first city:
{
  "city": "Srinagar",
  "attractions": [
    {
      "name": "Dal Lake",
      "num_reviews": "4329",
      "rating": "4.3",
      "ranking_position": "6",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "57",
          "name": "Nature & Parks"
        }
      ]
    },
    {
      "name": "Indira Gandhi Tulip Garden",
      "num_reviews": "1146",
      "rating": "4.6",
      "ranking_position": "2",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "57",
          "name": "Nature & Parks"
        }
      ]
    },
    {
      "name": "Sonamarg Hill Station",
      "num_reviews": "1174",
      "rating": "4.4",
      "ranking_position": "4",
      "neighborhood_info": null,
      "description": "",
      "subcategory": [
        {
          "key": "57",
          "name": "Nature & Parks"
        }
      ]
    },
    {
     

In [15]:
import pandas as pd
from IPython.display import display

# Flatten into a DataFrame (one row per city)
df = pd.json_normalize(final_list, sep="_")
display(df)

# Save JSON
out_path = "combined_data_hotel_attraction_flight_final.json"
with open(out_path, "w") as f:
    json.dump(final_list, f, indent=2)

print("\u2705 Saved combined results to", out_path)

,city,attractions,flights,hotels_bayesian_review_score,hotels_popularity,hotels_price
0,Srinagar,"[{'name': 'Dal Lake', 'num_reviews': '4329', '...","[{'to': 'Leh', 'date': '2025-07-13', 'flights'...","[{'name': 'John's Homestay', 'price': 82.65}, ...","[{'name': 'Dilnaaz group of houseboats', 'pric...","[{'name': 'LalaApartment', 'price': 19.6}, {'n..."
1,Leh,[],"[{'to': 'Chandigarh', 'date': '2025-07-16', 'f...","[{'name': 'Hyatt Place Salt Lake City/Lehi', '...","[{'name': 'Hyatt Place Salt Lake City/Lehi', '...","[{'name': 'Motel 6 Lehi, UT - Thanksgiving Poi..."
2,Chandigarh,[],"[{'to': 'Shimla', 'date': '2025-07-18', 'fligh...","[{'name': 'Anchorage 42', 'price': 183.67}, {'...","[{'name': 'Hyatt Regency Chandigarh', 'price':...","[{'name': 'Hotel Golden Height2', 'price': 10...."
3,Shimla,[],"[{'to': 'Dehradun', 'date': '2025-07-20', 'fli...","[{'name': 'Wildflower Hall, An Oberoi Resort, ...",[{'name': 'Victory Hotel and Restaurant by GS ...,"[{'name': 'Staynest Mashobra with balcony', 'p..."
4,Dehradun,"[{'name': 'Forest Research Institute', 'num_re...","[{'to': 'Jaipur', 'date': '2025-07-22', 'fligh...","[{'name': 'Rootz', 'price': 70.25}, {'name': '...","[{'name': 'Treebo Kanopy Greens', 'price': 83....","[{'name': 'Ananda Vatika Homestay', 'price': 1..."
5,Jaipur,[],"[{'to': 'Delhi', 'date': '2025-07-25', 'flight...","[{'name': 'Home of the world', 'price': 106.27...",[{'name': 'Treebo Premium Baramasi By Jai Club...,"[{'name': 'Eco Family Hostel', 'price': 14.7},..."
6,Delhi,"[{'name': 'Qutub Minar', 'num_reviews': '13493...",[],"[{'name': 'Tatvamasi Homestay', 'price': 152.2...","[{'name': 'Hotel N Maharani', 'price': 132.8},...","[{'name': 'Hotel Aerotech Plaza', 'price': 17...."


✅ Saved combined results to combined_data_hotel_attraction_flight_final.json


In [16]:
# Generate the final itinerary to be displayed
narrative = llm_generate(inputs, trip_plan, final_list)

with open("generated_itinerary.txt", "w") as f:
    f.write(narrative)